<a href="https://colab.research.google.com/github/sid0nair/3D-CNN-/blob/main/nn_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy matplotlib pandas

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

test_df = pd.read_csv('/content/sample_data/mnist_test.csv')
train_df = pd.read_csv('/content/sample_data/mnist_train_small.csv')

print("Training data sample:")
print(train_df.head())

train_df.info()

print("\nTesting data sample:")
print(test_df.head())

Training data sample:
   6  0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  ...  0.581  0.582  0.583  \
0  5  0    0    0    0    0    0    0    0    0  ...      0      0      0   
1  7  0    0    0    0    0    0    0    0    0  ...      0      0      0   
2  9  0    0    0    0    0    0    0    0    0  ...      0      0      0   
3  5  0    0    0    0    0    0    0    0    0  ...      0      0      0   
4  2  0    0    0    0    0    0    0    0    0  ...      0      0      0   

   0.584  0.585  0.586  0.587  0.588  0.589  0.590  
0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0  

[5 rows x 785 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Columns: 785 entries, 6 to 0.590
dtypes: int64(785)
memory usage: 119.8 MB

Testing data sample:


In [8]:
data_train = train_df.to_numpy().T
y_train = data_train[0]
x_train = data_train[1:]
data_test = test_df.to_numpy().T
y_test = data_test[0]
x_test = data_test[1:]

In [15]:
print(x_train.shape)
print(y_train.shape)

(784, 19999)
(19999,)


##**Need a starting (random) w1,b1,w2,b2 which will then be trained to fit the data**

In [42]:
def init_params():
  w1 = np.random.randn(128, 784) * np.sqrt(2 / 784)
  w2 = np.random.randn(10, 128) * np.sqrt(2 / 128)
  b1 = np.random.rand(10,1) - 0.5
  b2 = np.random.rand(10,1) - 0.5
  return w1,b1,w2,b2

def ReLu(z):
  return np.maximum(z,0)

def SoftMax(z):
  return np.exp(z)/sum(np.exp(z))

def SoftMax_updated(z):
  exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))  # prevent overflow
  return exp_z / np.sum(exp_z, axis=0, keepdims=True)

def forward_propogation(w1,b1,w2,b2,x):
  z1 = w1.dot(x) + b1
  a1 = ReLu(z1)
  z2 = w2.dot(a1) + b2
  a2 = SoftMax_updated(z2)
  return z1,a1,z2,a2

##Recheck
def one_hot(y):
  one_hot_y = np.zeros((y.size,y.max() + 1))
  one_hot_y[np.arange(y.size),y] = 1
  one_hot_y = one_hot_y.T
  return one_hot_y
##

def deriv_ReLu(z):
  return z > 0

def back_propogation(z1,a1,z2,a2,w2,x,y):
  #Need to one hot encode y
  dz2 = a2 - one_hot(y)
  dw2 = 1 / y.size * dz2.dot(a1.T)
  db2 = 1 / y.size * np.sum(dz2, axis=1, keepdims=True)
  dz1 = w2.T.dot(dz2) * (z1 > 0)
  dw1 = 1 / y.size * dz1.dot(x.T)
  db1 = 1 / y.size * np.sum(dz1)
  return dw1,db1,dw2,db2

def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
  w1 = w1 - alpha * dw1
  b1 = b1 - alpha * db1
  w2 = w2 - alpha * dw2
  b2 = b2 - alpha * db2
  return w1,b1,w2,b2

In [23]:
def get_accuracy(predictions,y):
  print(predictions,y)
  return np.sum(predictions == y) / y.size

def get_predictions(a2):
  return np.argmax(a2,0)

In [38]:
def gradient_descent(x,y,alpha,iterations):
  w1,b1,w2,b2 = init_params()
  for i in range(iterations):
    z1,a1,z2,a2 = forward_propogation(w1,b1,w2,b2,x)
    dw1,db1,dw2,db2 = back_propogation(z1,a1,z2,a2,w2,x,y)
    w1,b1,w2,b2 = update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
    if i % 10 == 0:
      print("Iteration: ", i)
      print("Accuracy: ",get_accuracy(get_predictions(a2),y))
  return w1,b1,w2,b2

In [43]:
W1,B1,W2,B2 = gradient_descent(x_train,y_train,0.1,500)

ValueError: operands could not be broadcast together with shapes (128,19999) (10,1) 